# Получение предсказаний по модели 

## Импортируем необходимые библиотеки

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd, pyarrow
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import mlflow

from torch.utils.data import DataLoader
import boto3
import io
import torch

from pathlib import Path
from datetime import date, timedelta, datetime
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient
import ast
import joblib
from sqlalchemy import text

# --- Настройка путей и sys.path ---
# Добавляем корневую директорию проекта в sys.path для импорта кастомных модулей
PROJECT_ROOT = Path().cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.config import config
from src.logger import logger
from src.database import clickhouse_engine, postgres_engine, ipdr_engine    
from src.visualization import *
from src.predprocessing_lstm import _process_group, create_lstm_sequences_credit_scoring, convert_categorical_to_str, collate_fn
from src.base_models import LSTMCreditScoringDataset
from src.modeling_lstm import _predict_probs_from_loader


2025-11-10 11:01:49,063 | my_logger - INFO - ✅ PostgreSQL engine создан | /data/aturov/scoring/src/database.py:21
2025-11-10 11:01:49,088 | my_logger - INFO - ✅ ClickHouse engine создан | /data/aturov/scoring/src/database.py:36
2025-11-10 11:01:49,089 | my_logger - INFO - ✅ IPDR ClickHouse engine создан | /data/aturov/scoring/src/database.py:46


Configuration loaded successfully.


In [2]:
# --- Настройка устройства для вычислений ---.
device = torch.device("cuda")

logger.info(f"Using device: {device}")

2025-11-10 11:01:50,185 | my_logger - INFO - Using device: cuda | /tmp/ipykernel_3121082/134817763.py:4


## Mlflow


In [3]:
logger.info("MLflow client created and tracking URI set.")

mlflow.set_tracking_uri(config.mlflow_config.HOST_MLFLOW)


client = MlflowClient()

exp = client.get_experiment_by_name(config.mlflow_config.NAME_PROJECT)
if exp is None:
    # create_experiment поддерживает tags
    try:
        exp_id = client.create_experiment(config.mlflow_config.NAME_PROJECT, tags={"model": config.mlflow_config.NAME_MODEL_CLIENT})
    except Exception:
        # race: если кто-то создал эксперимент параллельно — получить его
        exp = client.get_experiment_by_name(config.mlflow_config.NAME_PROJECT)
        exp_id = exp.experiment_id if exp else None
    if exp_id:
        exp = client.get_experiment(exp_id)

# установить/обновить теги и описание (описание в UI хранится как тег "mlflow.note.content")
if exp is not None:
    client.set_experiment_tag(exp.experiment_id, "model", config.mlflow_config.NAME_MODEL_CLIENT)
    if config.mlflow_config.EXPERIMENT_DESCRIPTION:
        client.set_experiment_tag(exp.experiment_id, "mlflow.note.content", config.mlflow_config.EXPERIMENT_DESCRIPTION)

# сделать эксперимент активным
mlflow.set_experiment(config.mlflow_config.NAME_PROJECT)
exp_id = mlflow.get_experiment_by_name(config.mlflow_config.NAME_PROJECT).experiment_id


2025-11-10 11:01:50,225 | my_logger - INFO - MLflow client created and tracking URI set. | /tmp/ipykernel_3121082/2002293533.py:1


In [4]:
version_info = client.get_model_version_by_alias(name=config.mlflow_config.NAME_MODEL_CLIENT, alias='test')
run_id = version_info.run_id
logger.info(f"Найден run_id по алиасу 'test': {run_id}")
version_info


2025-11-10 11:01:50,488 | my_logger - INFO - Найден run_id по алиасу 'test': ec0536e42419435895f30950d76e651a | /tmp/ipykernel_3121082/693350137.py:3


<ModelVersion: aliases=['test'], creation_timestamp=1762402581652, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1762402581652, metrics=None, model_id=None, name='LSTM_Scoring', params=None, run_id='ec0536e42419435895f30950d76e651a', run_link='', source='s3://mlflow/3/models/m-79d7885beb054a37b9285232c681fc6b/artifacts', status='READY', status_message=None, tags={}, user_id='', version='6'>

In [5]:
optimal_threshold = client.get_run(run_id).data.params.get('optimal_threshold')
optimal_f1_threshold = client.get_run(run_id).data.params.get('optimal_f1_threshold')
optimal_f1_threshold, optimal_threshold

('0.4601399004459381', '0.29767337441444397')

In [6]:
SPLINT_DATE = client.get_run(run_id).data.params.get('split_date')
DATE_END = client.get_run(run_id).data.params.get('date_end')
NAME_DATAFRAME = client.get_run(run_id).data.params.get('name_dataframe')
COUNT_WEEKS = int(client.get_run(run_id).data.params.get('count_weeks'))
OVERDUE_DAYS_MAX = int(client.get_run(run_id).data.params.get('overdue_days_max'))
TOTAL_OVERDUE = int(client.get_run(run_id).data.params.get('total_overdue'))
NAME_DATAFRAME_WEEKS = f'{NAME_DATAFRAME}_{COUNT_WEEKS}_{OVERDUE_DAYS_MAX}_{TOTAL_OVERDUE}' # имя файла с признаками по неделям for LSTMs models
DATE_FEATURES = client.get_run(run_id).data.params.get('date_features')
KEYS_COLUMNS = ast.literal_eval(client.get_run(run_id).data.params.get('name_columns'))
CURRENT_DATE = datetime.now().strftime('%Y%m%d')


if CURRENT_DATE > SPLINT_DATE:
    logger.info(f"Текущая дата {CURRENT_DATE} больше даты сплита {SPLINT_DATE} - дата обучающей выборки")
else:
    logger.warning(f"Текущая дата {CURRENT_DATE} меньше даты сплита {SPLINT_DATE} - дата обучающей выборки")

2025-11-10 11:01:50,723 | my_logger - INFO - Текущая дата 20251110 больше даты сплита 2024-12-01 - дата обучающей выборки | /tmp/ipykernel_3121082/125430703.py:14


In [7]:
NAME_DATAFRAME

'features_weeks'

In [8]:
s3_client = boto3.client(
    's3',
    endpoint_url=config.mlflow_config.MINIO_ENDPOINT,
    aws_access_key_id=config.mlflow_config.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=config.mlflow_config.AWS_SECRET_ACCESS_KEY,
)

def load_from_s3(bucket, key):
    '''Загружает файл из S3 и возвращает его как BytesIO объект'''
    response = s3_client.get_object(Bucket=bucket, Key=key)
    data = response['Body'].read()
    return io.BytesIO(data)

# Загрузка файлов напрямую
prefix = f"{exp_id}/{run_id}/artifacts/preprocessor"

cat_maps = joblib.load(load_from_s3(config.mlflow_config.BUCKET_NAME, f"{prefix}/cat_maps.pkl"))
preprocessor = joblib.load(load_from_s3(config.mlflow_config.BUCKET_NAME, f"{prefix}/preprocessor_lstm_{DATE_FEATURES}.joblib"))

numeric_cols = joblib.load(load_from_s3(config.mlflow_config.BUCKET_NAME, f"{prefix}/numeric_cols.pkl"))
categorical_cols = joblib.load(load_from_s3(config.mlflow_config.BUCKET_NAME, f"{prefix}/categorical_cols.pkl"))

logger.info(f"Numeric cols: {len(numeric_cols)}, categorical cols: {len(categorical_cols)}")

2025-11-10 11:01:50,876 | my_logger - INFO - Numeric cols: 16, categorical cols: 4 | /tmp/ipykernel_3121082/270943303.py:23


In [9]:
model_key = version_info.source.replace(f's3://{config.mlflow_config.BUCKET_NAME}/', '') + '/data/model.pth'

response = s3_client.get_object(Bucket=config.mlflow_config.BUCKET_NAME, Key=model_key)
model_buffer = io.BytesIO(response['Body'].read())
model = torch.load(model_buffer, map_location=device, weights_only=False)
logger.info(f"Loaded model: {model}")

/data/aturov/scoring/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:827: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2025-11-10 11:01:51,118 | my_logger - INFO - Loaded model: LSTMModel(
  (embs): ModuleList(
    (0): Embedding(5, 2)
    (1): Embedding(6, 2)
    (2): Embedding(13, 3)
    (3): Embedding(5, 2)
  )
  (lstm): LSTM(25, 128, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
) | /tmp/ipykernel_3121082/1697114400.py:6


In [10]:
KEYS_COLUMNS

['days_from_dt_end_to_price_change_date',
 'FLAG_DEVICE_4G',
 'days_from_dt_end_to_date_lad',
 'USAGE_INTERNET_NIGHT',
 'ACTIVE_IND',
 'REGION_CELL',
 'days_from_dt_end_to_act_date',
 'REVENUE_ABONKA',
 'GENDER',
 'BALANCE_END',
 'USAGE_INTERNET_LTE',
 'COUNT_RECHARGE',
 'LIFETIME_TOTAL',
 'USAGE_ABONKA_TP',
 'INTERCONNECT_MN_IN',
 'USAGE_INTERNET_3G_FREE',
 'days_from_dt_end_to_date_contract',
 'USAGE_NUM_INTERNET_PAK',
 'REVENUE_INTERNET_PAYG',
 'USAGE_OUT_INT_VOICE_RUSSIA']

## Загрузка клиентов и фичей для предсказания

In [11]:
def make_query(engine, number_weeks):
    """
    Возвращает только нужные KEYS_COLUMNS из DWH.dm_datamart_weekly.
    Для days_from_dt_end_to_* считаем: dateDiff('day', <DATE_COL>, addDays(DT, 7))
    """
    computed = {
        'days_from_dt_end_to_price_change_date': "coalesce(dateDiff('day', PRICE_CHANGE_DATE, addDays(DT, 7)), -1) AS days_from_dt_end_to_price_change_date",
        'days_from_dt_end_to_act_date':          "coalesce(dateDiff('day', ACT_DATE,          addDays(DT, 7)), -1) AS days_from_dt_end_to_act_date",
        'days_from_dt_end_to_date_contract':     "coalesce(dateDiff('day', DATE_CONTRACT,     addDays(DT, 7)), -1) AS days_from_dt_end_to_date_contract",
        'days_from_dt_end_to_date_lad':          "coalesce(dateDiff('day', DATE_LAD,          addDays(DT, 7)), -1) AS days_from_dt_end_to_date_lad",
        'days_from_dt_end_to_date_inactive':      "coalesce(dateDiff('day', DATE_INACTIVE,     addDays(DT, 7)), -1) AS days_from_dt_end_to_date_inactive",
        'days_from_dt_end_to_date_abonka':        "coalesce(dateDiff('day', DATE_ABONKA,       addDays(DT, 7)), -1) AS days_from_dt_end_to_date_abonka",
    
    }

    select_items = []
    for col in KEYS_COLUMNS:  # KEYS_COLUMNS уже загружается выше из MLflow
        if col in computed:
            select_items.append(computed[col])
        else:
            select_items.append(col)
    select_items.append('CTN as ctn, SUBS_ID as subs_id')  # Добавляем колонку DT для фильтрации по дате
    select_clause = ",\n                ".join(select_items)
    query = f"""
        SELECT
                {select_clause}
        FROM DWH.dm_datamart_weekly w
        where w.DT <= toDate('{CURRENT_DATE}')
          AND w.DT = toStartOfWeek('{CURRENT_DATE}' - INTERVAL {number_weeks} WEEK - INTERVAL 1 DAY, 1) 
          limit 10000
    """

    df = pd.read_sql(query, engine)
    logger.info(f"Loaded data for {number_weeks} weeks: {df.shape}")
    return df

In [12]:
df_features_parts = []
for number_week in range(1, COUNT_WEEKS+1):  # от 1 до 12 недель включительно
    logger.info(f"COUNT_WEEKS = {number_week}")
    df_part = make_query(clickhouse_engine, number_weeks=number_week)
    if df_part is None or df_part.empty:
        logger.warning(f"COUNT_WEEKS = {number_week}, пустой датафрейм, пропускаем")
        continue
    df_part[config.features.SEQ_COL] = number_week
    logger.info(f"COUNT_WEEKS = {number_week}, shape = {df_part.shape}")
    df_features_parts.append(df_part)
df = pd.concat(df_features_parts, ignore_index=True)
logger.info(f"Final features shape: {df.shape}")
df.head(3)

2025-11-10 11:01:51,214 | my_logger - INFO - COUNT_WEEKS = 1 | /tmp/ipykernel_3121082/115029470.py:3


2025-11-10 11:01:51,505 | my_logger - INFO - Loaded data for 1 weeks: (10000, 22) | /tmp/ipykernel_3121082/1495415186.py:34
2025-11-10 11:01:51,507 | my_logger - INFO - COUNT_WEEKS = 1, shape = (10000, 23) | /tmp/ipykernel_3121082/115029470.py:9
2025-11-10 11:01:51,507 | my_logger - INFO - COUNT_WEEKS = 2 | /tmp/ipykernel_3121082/115029470.py:3
2025-11-10 11:01:51,723 | my_logger - INFO - Loaded data for 2 weeks: (10000, 22) | /tmp/ipykernel_3121082/1495415186.py:34
2025-11-10 11:01:51,724 | my_logger - INFO - COUNT_WEEKS = 2, shape = (10000, 23) | /tmp/ipykernel_3121082/115029470.py:9
2025-11-10 11:01:51,725 | my_logger - INFO - COUNT_WEEKS = 3 | /tmp/ipykernel_3121082/115029470.py:3
2025-11-10 11:01:51,942 | my_logger - INFO - Loaded data for 3 weeks: (10000, 22) | /tmp/ipykernel_3121082/1495415186.py:34
2025-11-10 11:01:51,943 | my_logger - INFO - COUNT_WEEKS = 3, shape = (10000, 23) | /tmp/ipykernel_3121082/115029470.py:9
2025-11-10 11:01:51,943 | my_logger - INFO - COUNT_WEEKS = 4

,days_from_dt_end_to_price_change_date,FLAG_DEVICE_4G,days_from_dt_end_to_date_lad,USAGE_INTERNET_NIGHT,ACTIVE_IND,REGION_CELL,days_from_dt_end_to_act_date,REVENUE_ABONKA,GENDER,BALANCE_END,...,USAGE_ABONKA_TP,INTERCONNECT_MN_IN,USAGE_INTERNET_3G_FREE,days_from_dt_end_to_date_contract,USAGE_NUM_INTERNET_PAK,REVENUE_INTERNET_PAYG,USAGE_OUT_INT_VOICE_RUSSIA,ctn,subs_id,count_weeks
0,83,,-1,0.0,0,,83,0.0,,0.0,...,0,0.0,0.0,83,0,0.0,0.0,996774049151,21590845,1
1,83,,-1,0.0,0,,83,0.0,,0.0,...,0,0.0,0.0,83,0,0.0,0.0,996774049152,21590848,1
2,83,,-1,0.0,0,,83,0.0,,0.0,...,0,0.0,0.0,83,0,0.0,0.0,996774049158,21590866,1


In [13]:
# Получаем уникальные subs_id с count_weeks == 1 и оставляем только их для предсказаний
sub_ids = df.query(f"{config.features.SEQ_COL} == 1")['subs_id'].unique()
df = df[df['subs_id'].isin(sub_ids)].reset_index(drop=True)
logger.info(f"After filtering by subs_id with count_weeks == 1, shape: {df.shape}")
df.set_index(['subs_id', 'ctn', config.features.SEQ_COL], inplace=True)
df.head(3)

2025-11-10 11:01:54,143 | my_logger - INFO - After filtering by subs_id with count_weeks == 1, shape: (10000, 23) | /tmp/ipykernel_3121082/504841215.py:4


,,,days_from_dt_end_to_price_change_date,FLAG_DEVICE_4G,days_from_dt_end_to_date_lad,USAGE_INTERNET_NIGHT,ACTIVE_IND,REGION_CELL,days_from_dt_end_to_act_date,REVENUE_ABONKA,GENDER,BALANCE_END,USAGE_INTERNET_LTE,COUNT_RECHARGE,LIFETIME_TOTAL,USAGE_ABONKA_TP,INTERCONNECT_MN_IN,USAGE_INTERNET_3G_FREE,days_from_dt_end_to_date_contract,USAGE_NUM_INTERNET_PAK,REVENUE_INTERNET_PAYG,USAGE_OUT_INT_VOICE_RUSSIA
subs_id,ctn,count_weeks,,,,,,,,,,,,,,,,,,,,
21590845,996774049151,1,83,,-1,0.0,0,,83,0.0,,0.0,0.0,0,83,0,0.0,0.0,83,0,0.0,0.0
21590848,996774049152,1,83,,-1,0.0,0,,83,0.0,,0.0,0.0,0,83,0,0.0,0.0,83,0,0.0,0.0
21590866,996774049158,1,83,,-1,0.0,0,,83,0.0,,0.0,0.0,0,83,0,0.0,0.0,83,0,0.0,0.0


## Преобразование данных для предсказания

In [14]:
categorical_cols

['FLAG_DEVICE_4G', 'ACTIVE_IND', 'REGION_CELL', 'GENDER']

In [15]:
val_tab_full = convert_categorical_to_str(df, categorical_cols)
# Определяем ID колонки для группировки
id_cols_from_index = list(val_tab_full.index.names)
id_cols = [col for col in id_cols_from_index if col not in [config.features.SEQ_COL, config.features.TARGET_COL]]
logger.info(f"ID columns for grouping: {id_cols}")

2025-11-10 11:01:54,227 | my_logger - INFO - Categorical columns converted to str | /data/aturov/scoring/src/predprocessing_lstm.py:90
2025-11-10 11:01:54,228 | my_logger - INFO - ID columns for grouping: ['subs_id', 'ctn'] | /tmp/ipykernel_3121082/734171255.py:5


In [16]:
# Создаем последовательности для validation
Xn_val, Xc_val, y_val_seq, val_metadata, val_len = create_lstm_sequences_credit_scoring(
    df=val_tab_full,
    id_cols=id_cols,
    numeric_cols=numeric_cols,
    categorical_cols=categorical_cols,
    cat_maps=cat_maps,
    scaler=preprocessor, 
    seq_col=config.features.SEQ_COL,
    target_col=config.features.TARGET_COL,
    only_prediction=True
)

2025-11-10 11:01:54,891 | my_logger - INFO - Найдено 10000 уникальных групп для обработки на нескольких ядрах. | /data/aturov/scoring/src/predprocessing_lstm.py:63
Создание последовательностей:   1%|          | 64/10000 [00:00<00:36, 272.74it/s]

Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.Configuration loaded successfully.

Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.Configuration loaded successfully.

Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded

Создание последовательностей:   2%|▏         | 192/10000 [00:02<02:05, 78.19it/s]

Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.


Создание последовательностей:   4%|▍         | 384/10000 [00:02<00:46, 205.78it/s]

Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.
Configuration loaded successfully.


Создание последовательностей: 100%|██████████| 10000/10000 [00:06<00:00, 1574.12it/s]
2025-11-10 11:02:01,754 | my_logger - INFO - Созданы списки последовательностей: 10000 шт. | /data/aturov/scoring/src/predprocessing_lstm.py:79


In [17]:
val_dataset = LSTMCreditScoringDataset(Xn_val, Xc_val, y_val_seq, val_len, val_metadata)
logger.info(f"Val dataset: {len(val_dataset)} samples")
# ← Проверяем типы данных в батче
sample_batch = next(iter(DataLoader(val_dataset, batch_size=4)))
logger.info(f"Sample batch dtypes:")
logger.info(f"  X_num dtype: {sample_batch['X_num'].dtype}")
logger.info(f"  X_cat dtype: {sample_batch['X_cat'].dtype}")
logger.info(f"  y dtype: {sample_batch['y'].dtype}")
logger.info(f"  y values: {sample_batch['y']}")

va_loader_pred = DataLoader(
    val_dataset,
    batch_size=config.lstm.BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    pin_memory=True
)

logger.info(f"Val loader: {len(va_loader_pred)} batches")

2025-11-10 11:02:01,815 | my_logger - INFO - Val dataset: 10000 samples | /tmp/ipykernel_3121082/538504923.py:2
2025-11-10 11:02:01,818 | my_logger - INFO - Sample batch dtypes: | /tmp/ipykernel_3121082/538504923.py:5
2025-11-10 11:02:01,819 | my_logger - INFO -   X_num dtype: torch.float32 | /tmp/ipykernel_3121082/538504923.py:6
2025-11-10 11:02:01,819 | my_logger - INFO -   X_cat dtype: torch.int64 | /tmp/ipykernel_3121082/538504923.py:7
2025-11-10 11:02:01,819 | my_logger - INFO -   y dtype: torch.float32 | /tmp/ipykernel_3121082/538504923.py:8
2025-11-10 11:02:01,821 | my_logger - INFO -   y values: tensor([0., 0., 0., 0.]) | /tmp/ipykernel_3121082/538504923.py:9
2025-11-10 11:02:01,822 | my_logger - INFO - Val loader: 157 batches | /tmp/ipykernel_3121082/538504923.py:19


In [18]:
# Перед инференсом
model.eval()

# Функция для получения предсказаний вероятностей из DataLoader
val_probs   = _predict_probs_from_loader(va_loader_pred, model, device)
assert len(val_probs) == len(val_dataset), "Количество вероятностей не совпадает с числом примеров"

In [21]:
meta_df_valid = val_dataset.metadata.copy()
meta_df_valid = meta_df_valid.reset_index(drop=True)
meta_df_valid['probability'] = val_probs.astype(float)
# Предсказанные вероятности действуют с insert_datetime == CURRENT_DATE по  CURRENT_DATE + 7 дней
meta_df_valid['insert_datetime'] = pd.to_datetime(CURRENT_DATE)
meta_df_valid['observation_period_end_date'] = (pd.to_datetime(CURRENT_DATE) + pd.Timedelta(days=7)).date()  # date
meta_df_valid['load_dt'] = pd.Timestamp.utcnow()

# приведение типов под схему таблицы
meta_df_valid['subs_id'] = meta_df_valid['subs_id'].astype('int64')
meta_df_valid['ctn'] = meta_df_valid['ctn'].astype('int64')

records = meta_df_valid[['subs_id','ctn','insert_datetime','probability',
                         'observation_period_end_date','load_dt']].to_dict('records')
len(records)

10000

## Загрузка предсказаний из модели в БД

In [ ]:
from sqlalchemy.orm import sessionmaker
from src.base_models import InsertPredictionsScoring, Base  # класс уже описан тут: src/base_models.py

# создать таблицу, если её ещё нет
Base.metadata.create_all(clickhouse_engine)

Session = sessionmaker(bind=clickhouse_engine)
session = Session()

In [24]:
# быстрая массовая вставка
session.execute(InsertPredictionsScoring.__table__.insert(), records)
session.commit()

# проверка
check_df = pd.read_sql(
    "SELECT subs_id, ctn, probability, insert_datetime, observation_period_end_date "
    "FROM data_science.credit_scoring_predictions "
    f"WHERE insert_datetime = toDate('{CURRENT_DATE}') "
    "ORDER BY insert_datetime DESC LIMIT 5",
    clickhouse_engine
)
check_df

,subs_id,ctn,probability,insert_datetime,observation_period_end_date
0,21694642,996774064937,0.474604,2025-11-10,2025-11-17
1,21694615,996774064927,0.474604,2025-11-10,2025-11-17
2,21694621,996774064929,0.474604,2025-11-10,2025-11-17
3,21694609,996774064925,0.474604,2025-11-10,2025-11-17
4,21694639,996774064936,0.474604,2025-11-10,2025-11-17
